In [1]:
import cv2

In [1]:
import os
import shutil
import cv2
import numpy as np
import matplotlib.pyplot as plt

### 설명 <br>
- 본 코드는 https://m.blog.naver.com/rhrkdfus/221531159811 와 https://learnopencv.com/deep-learning-based-human-pose-estimation-using-opencv-cpp-python/, https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html 를 참고해 만들었습니다.<br>
- 맨 아래 cell(test code 바로 위의)에서 하는 일은 다음과 같습니다.<br>
1. *training_dir* 는 training data가 있는 디랙토리입니다. 여기서 *folder*를 순회하면서<br>
2. 확장자가 .txt일 때는 무시하고, 아닐 경우에는 *folder*의 *image*를 순회합니다.<br>
3. 이 *image*를 frame_coco = cv2.imread(image_dir) 와 같은 형식으로 받아, coco 모델에 넘겨줍니다. coco 모델이 무엇인지는 아직은 잘 모르겠습니다만, Openpose realease 파일에 같이 있는 것을 보니 비슷한 종류의 모델인 것 같습니다. <br>
4. *output_keypoints*에서 *points =  []* 를 (x, y)의 형식으로 채웁니다. 이것을 *BODY_PARTS_COCO* 와 비교해 이미지마다 필요한 keypoint index만 따서, delimiter를 중간에 넣어준 다음 텍스트 폴더의 한 줄로 만듭니다. <br><br>

더불어 openpose model을 받으면 model net이 저장되는 디랙토리가 있습니다(protoFile_coco, weightsFile_coco). 이것과 keypoint를 뽑고자 하는 데이터를 가리키는 디랙토리(training_dir)만 각자 컴퓨터에 맞게 변경해주시면 됩니다. 

In [34]:
def output_keypoints(frame, proto_file, weights_file, threshold, model_name, BODY_PARTS):
    global points

    # 네트워크 불러오기
    net = cv2.dnn.readNetFromCaffe(proto_file, weights_file)

    # 입력 이미지의 사이즈 정의
    image_height = 260
    image_width = 210

    # 네트워크에 넣기 위한 전처리
    input_blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (image_width, image_height), (0, 0, 0), swapRB=False, crop=False)

    # 전처리된 blob 네트워크에 입력
    net.setInput(input_blob)

    # 결과 받아오기
    out = net.forward()
    # The output is a 4D matrix :
    # The first dimension being the image ID ( in case you pass more than one image to the network ).
    # The second dimension indicates the index of a keypoint.
    # The model produces Confidence Maps and Part Affinity maps which are all concatenated.
    # For COCO model it consists of 57 parts – 18 keypoint confidence Maps + 1 background + 19*2 Part Affinity Maps. Similarly, for MPI, it produces 44 points.
    # We will be using only the first few points which correspond to Keypoints.
    # The third dimension is the height of the output map.
    out_height = out.shape[2]
    # The fourth dimension is the width of the output map.
    out_width = out.shape[3]

    # 원본 이미지의 높이, 너비를 받아오기
    frame_height, frame_width = frame.shape[:2]

    # 포인트 리스트 초기화
    points = []

    print(f"\n============================== {model_name} Model ==============================")
    for i in range(len(BODY_PARTS)):

        # 신체 부위의 confidence map
        prob_map = out[0, i, :, :]

        # 최소값, 최대값, 최소값 위치, 최대값 위치
        min_val, prob, min_loc, point = cv2.minMaxLoc(prob_map)

        # 원본 이미지에 맞게 포인트 위치 조정
        x = (frame_width * point[0]) / out_width
        x = int(x)
        y = (frame_height * point[1]) / out_height
        y = int(y)

        if prob > threshold:  # [pointed]
            cv2.circle(frame, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1, lineType=cv2.LINE_AA)

            points.append((x, y))
            print(f"[pointed] {BODY_PARTS[i]} ({i}) => prob: {prob:.5f} / x: {x} / y: {y}")

        else:  # [not pointed]
            cv2.circle(frame, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, lineType=cv2.LINE_AA)

            points.append(None)
            print(f"[not pointed] {BODY_PARTS[i]} ({i}) => prob: {prob:.5f} / x: {x} / y: {y}")

    #cv2.imshow("Output_Keypoints", frame)
    #cv2.waitKey(0)
    return frame

In [33]:
def output_keypoints_with_lines(frame, POSE_PAIRS):
    print()
    for pair in POSE_PAIRS:
        part_a = pair[0]  # 0 (Head)
        part_b = pair[1]  # 1 (Neck)
        if points[part_a] and points[part_b]:
            print(f"[linked] {part_a} {points[part_a]} <=> {part_b} {points[part_b]}")
            cv2.line(frame, points[part_a], points[part_b], (0, 255, 0), 3)
        else:
            print(f"[not linked] {part_a} {points[part_a]} <=> {part_b} {points[part_b]}")

    #cv2.imshow("output_keypoints_with_lines", frame)
    #cv2.waitKey(0)
    cv2.destroyAllWindows()

In [51]:
# COCO model body parts // what is coco? is it different from OPENPOSE?
BODY_PARTS_COCO = {0: "Nose", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
                   5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "RHip", 9: "RKnee",
                   10: "RAnkle", 11: "LHip", 12: "LKnee", 13: "LAnkle", 14: "REye",
                   15: "LEye", 16: "REar", 17: "LEar", 18: "Background"}

POSE_PAIRS_COCO = [[0, 1], [0, 14], [0, 15], [1, 2], [1, 5], [1, 8], [1, 11], [2, 3], [3, 4],
                   [5, 6], [6, 7], [8, 9], [9, 10], [12, 13], [11, 12], [14, 16], [15, 17]]
point_indices = [0,1,2,3,4,5,6,7,14,15,16,17] # Select face, arm features only. 

###################### Output text file config ###########################
# Contents :       image number    (x0, y0) (x1, y1)                     #
# Generated path : The master image bunch directory(2014T)               #
# Delimiter :      folder/file : '    ', keypoint coordinate : '  '      #
# Each keypoint    corresponds to keys in point_indices in BODY_PARTS    #
##########################################################################

protoFile_coco = "C:/Users/PC/2021-MLVU/openpose-master/openpose-master/models/pose/coco/pose_deploy_linevec.prototxt"
weightsFile_coco = "C:/Users/PC/2021-MLVU/openpose-master/openpose-master/models/pose/coco/pose_iter_440000.caffemodel"

point_dict = {}
training_dir = "C:/Users/PC/2021-MLVU/openpose-test/2014T_test/"

for folder in os.listdir(training_dir):
    imgs_dir = training_dir+folder
    print(imgs_dir)
    
    f = open(imgs_dir + '.txt', 'w')
    
    for image in os.listdir(imgs_dir):
        if image.split('.')[-1] == 'txt':
            pass
        else:
            print(image)
            image_dir = imgs_dir+'/'+image
            points = []                        # keypoints

            frame_coco = cv2.imread(image_dir)     # Images to read by coco

            frame_COCO = output_keypoints(frame=frame_coco, proto_file=protoFile_coco, weights_file=weightsFile_coco,
                                     threshold=0.2, model_name="COCO", BODY_PARTS=BODY_PARTS_COCO)
            output_keypoints_with_lines(frame=frame_COCO, POSE_PAIRS=POSE_PAIRS_COCO)

            pointlist = list(str(points[i]) for i in point_indices)
            pointstring = "  ".join(pointlist)
            point_dict[image] = pointstring
        
        f.write(image.split('.')[0][-4:] + '    ' + pointstring+'\n')
    f.close()


C:/Users/PC/2021-MLVU/openpose-test/2014T_test/100830_tagesschau
images0001.png

============================== COCO Model ==============================
[pointed] Nose (0) => prob: 0.77883 / x: 85 / y: 55
[pointed] Neck (1) => prob: 0.80159 / x: 85 / y: 118
[pointed] RShoulder (2) => prob: 0.66614 / x: 46 / y: 118
[pointed] RElbow (3) => prob: 0.66124 / x: 31 / y: 181
[pointed] RWrist (4) => prob: 0.66738 / x: 31 / y: 204
[pointed] LShoulder (5) => prob: 0.71035 / x: 124 / y: 118
[pointed] LElbow (6) => prob: 0.61047 / x: 147 / y: 189
[pointed] LWrist (7) => prob: 0.39301 / x: 124 / y: 236
[pointed] RHip (8) => prob: 0.29966 / x: 54 / y: 244
[not pointed] RKnee (9) => prob: 0.00464 / x: 7 / y: 252
[not pointed] RAnkle (10) => prob: 0.00255 / x: 23 / y: 189
[pointed] LHip (11) => prob: 0.31052 / x: 108 / y: 244
[not pointed] LKnee (12) => prob: 0.01490 / x: 116 / y: 252
[not pointed] LAnkle (13) => prob: 0.00294 / x: 124 / y: 236
[pointed] REye (14) => prob: 0.81085 / x: 70 / y: 47
[po


============================== COCO Model ==============================
[pointed] Nose (0) => prob: 0.76630 / x: 85 / y: 47
[pointed] Neck (1) => prob: 0.72870 / x: 85 / y: 118
[pointed] RShoulder (2) => prob: 0.61036 / x: 46 / y: 126
[pointed] RElbow (3) => prob: 0.69826 / x: 23 / y: 204
[pointed] RWrist (4) => prob: 0.58516 / x: 31 / y: 149
[pointed] LShoulder (5) => prob: 0.73666 / x: 124 / y: 118
[pointed] LElbow (6) => prob: 0.65267 / x: 147 / y: 189
[pointed] LWrist (7) => prob: 0.52750 / x: 124 / y: 244
[pointed] RHip (8) => prob: 0.28417 / x: 62 / y: 252
[not pointed] RKnee (9) => prob: 0.00294 / x: 23 / y: 204
[not pointed] RAnkle (10) => prob: 0.00228 / x: 77 / y: 70
[pointed] LHip (11) => prob: 0.31215 / x: 116 / y: 252
[not pointed] LKnee (12) => prob: 0.01558 / x: 116 / y: 252
[not pointed] LAnkle (13) => prob: 0.00498 / x: 116 / y: 252
[pointed] REye (14) => prob: 0.87003 / x: 77 / y: 47
[pointed] LEye (15) => prob: 0.85322 / x: 93 / y: 39
[pointed] REar (16) => prob: 0


============================== COCO Model ==============================
[pointed] Nose (0) => prob: 0.71852 / x: 85 / y: 47
[pointed] Neck (1) => prob: 0.75243 / x: 85 / y: 118
[pointed] RShoulder (2) => prob: 0.64607 / x: 54 / y: 118
[pointed] RElbow (3) => prob: 0.66210 / x: 23 / y: 212
[pointed] RWrist (4) => prob: 0.63535 / x: 38 / y: 173
[pointed] LShoulder (5) => prob: 0.75600 / x: 124 / y: 118
[pointed] LElbow (6) => prob: 0.62923 / x: 140 / y: 189
[pointed] LWrist (7) => prob: 0.66554 / x: 124 / y: 220
[pointed] RHip (8) => prob: 0.34421 / x: 54 / y: 244
[not pointed] RKnee (9) => prob: 0.00363 / x: 31 / y: 212
[not pointed] RAnkle (10) => prob: 0.00218 / x: 15 / y: 252
[pointed] LHip (11) => prob: 0.33724 / x: 108 / y: 244
[not pointed] LKnee (12) => prob: 0.00276 / x: 116 / y: 252
[not pointed] LAnkle (13) => prob: 0.00256 / x: 85 / y: 39
[pointed] REye (14) => prob: 0.82830 / x: 77 / y: 39
[pointed] LEye (15) => prob: 0.85709 / x: 93 / y: 39
[pointed] REar (16) => prob: 0.


============================== COCO Model ==============================
[pointed] Nose (0) => prob: 0.67310 / x: 85 / y: 47
[pointed] Neck (1) => prob: 0.77522 / x: 85 / y: 118
[pointed] RShoulder (2) => prob: 0.63603 / x: 46 / y: 118
[pointed] RElbow (3) => prob: 0.64713 / x: 15 / y: 189
[pointed] RWrist (4) => prob: 0.57048 / x: 15 / y: 126
[pointed] LShoulder (5) => prob: 0.62122 / x: 132 / y: 118
[pointed] LElbow (6) => prob: 0.52490 / x: 147 / y: 204
[pointed] LWrist (7) => prob: 0.42354 / x: 140 / y: 173
[pointed] RHip (8) => prob: 0.30330 / x: 62 / y: 252
[not pointed] RKnee (9) => prob: 0.00227 / x: 7 / y: 196
[not pointed] RAnkle (10) => prob: 0.00210 / x: 77 / y: 55
[pointed] LHip (11) => prob: 0.25879 / x: 116 / y: 244
[not pointed] LKnee (12) => prob: 0.00614 / x: 124 / y: 212
[not pointed] LAnkle (13) => prob: 0.00184 / x: 85 / y: 39
[pointed] REye (14) => prob: 0.85846 / x: 77 / y: 47
[pointed] LEye (15) => prob: 0.79530 / x: 93 / y: 39
[pointed] REar (16) => prob: 0.76


============================== COCO Model ==============================
[pointed] Nose (0) => prob: 0.88903 / x: 85 / y: 55
[pointed] Neck (1) => prob: 0.77298 / x: 85 / y: 118
[pointed] RShoulder (2) => prob: 0.63288 / x: 54 / y: 118
[pointed] RElbow (3) => prob: 0.77426 / x: 15 / y: 181
[pointed] RWrist (4) => prob: 0.55589 / x: 31 / y: 141
[pointed] LShoulder (5) => prob: 0.71082 / x: 124 / y: 118
[pointed] LElbow (6) => prob: 0.75186 / x: 147 / y: 204
[pointed] LWrist (7) => prob: 0.52072 / x: 124 / y: 173
[pointed] RHip (8) => prob: 0.31845 / x: 54 / y: 252
[not pointed] RKnee (9) => prob: 0.00290 / x: 15 / y: 189
[not pointed] RAnkle (10) => prob: 0.00169 / x: 15 / y: 133
[pointed] LHip (11) => prob: 0.25780 / x: 108 / y: 252
[not pointed] LKnee (12) => prob: 0.00535 / x: 116 / y: 204
[not pointed] LAnkle (13) => prob: 0.00153 / x: 23 / y: 157
[pointed] REye (14) => prob: 0.91355 / x: 77 / y: 47
[pointed] LEye (15) => prob: 0.84462 / x: 93 / y: 47
[pointed] REar (16) => prob: 0

...


============================== COCO Model ==============================
[pointed] Nose (0) => prob: 0.80812 / x: 70 / y: 70
[pointed] Neck (1) => prob: 0.76937 / x: 77 / y: 126
[pointed] RShoulder (2) => prob: 0.67109 / x: 38 / y: 133
[pointed] RElbow (3) => prob: 0.71864 / x: 31 / y: 212
[pointed] RWrist (4) => prob: 0.64896 / x: 62 / y: 141
[pointed] LShoulder (5) => prob: 0.63783 / x: 116 / y: 126
[pointed] LElbow (6) => prob: 0.74852 / x: 147 / y: 212
[pointed] LWrist (7) => prob: 0.63166 / x: 108 / y: 157
[not pointed] RHip (8) => prob: 0.16794 / x: 62 / y: 252
[not pointed] RKnee (9) => prob: 0.00378 / x: 38 / y: 220
[not pointed] RAnkle (10) => prob: 0.00272 / x: 62 / y: 118
[not pointed] LHip (11) => prob: 0.14221 / x: 116 / y: 252
[not pointed] LKnee (12) => prob: 0.00296 / x: 23 / y: 39
[not pointed] LAnkle (13) => prob: 0.00481 / x: 85 / y: 110
[pointed] REye (14) => prob: 0.79311 / x: 62 / y: 63
[pointed] LEye (15) => prob: 0.84048 / x: 77 / y: 63
[not pointed] REar (16) 


============================== COCO Model ==============================
[pointed] Nose (0) => prob: 0.78139 / x: 70 / y: 63
[pointed] Neck (1) => prob: 0.76105 / x: 77 / y: 126
[pointed] RShoulder (2) => prob: 0.61531 / x: 46 / y: 126
[pointed] RElbow (3) => prob: 0.71867 / x: 31 / y: 220
[pointed] RWrist (4) => prob: 0.62625 / x: 62 / y: 149
[pointed] LShoulder (5) => prob: 0.70641 / x: 116 / y: 118
[pointed] LElbow (6) => prob: 0.77417 / x: 147 / y: 212
[pointed] LWrist (7) => prob: 0.69261 / x: 108 / y: 157
[not pointed] RHip (8) => prob: 0.13169 / x: 62 / y: 252
[not pointed] RKnee (9) => prob: 0.00320 / x: 38 / y: 220
[not pointed] RAnkle (10) => prob: 0.00312 / x: 62 / y: 118
[not pointed] LHip (11) => prob: 0.11653 / x: 116 / y: 252
[not pointed] LKnee (12) => prob: 0.00192 / x: 155 / y: 204
[not pointed] LAnkle (13) => prob: 0.00306 / x: 62 / y: 118
[pointed] REye (14) => prob: 0.86252 / x: 62 / y: 55
[pointed] LEye (15) => prob: 0.88913 / x: 77 / y: 55
[pointed] REar (16) =>


============================== COCO Model ==============================
[pointed] Nose (0) => prob: 0.72099 / x: 70 / y: 63
[pointed] Neck (1) => prob: 0.74574 / x: 85 / y: 126
[pointed] RShoulder (2) => prob: 0.64046 / x: 46 / y: 126
[pointed] RElbow (3) => prob: 0.72631 / x: 31 / y: 228
[pointed] RWrist (4) => prob: 0.49513 / x: 38 / y: 157
[pointed] LShoulder (5) => prob: 0.56194 / x: 124 / y: 126
[pointed] LElbow (6) => prob: 0.59813 / x: 140 / y: 220
[pointed] LWrist (7) => prob: 0.47245 / x: 140 / y: 149
[pointed] RHip (8) => prob: 0.23428 / x: 62 / y: 252
[not pointed] RKnee (9) => prob: 0.00233 / x: 46 / y: 244
[not pointed] RAnkle (10) => prob: 0.00132 / x: 93 / y: 63
[pointed] LHip (11) => prob: 0.21177 / x: 116 / y: 252
[not pointed] LKnee (12) => prob: 0.01010 / x: 124 / y: 220
[not pointed] LAnkle (13) => prob: 0.00188 / x: 132 / y: 212
[pointed] REye (14) => prob: 0.85897 / x: 62 / y: 55
[pointed] LEye (15) => prob: 0.89433 / x: 85 / y: 55
[not pointed] REar (16) => pro


============================== COCO Model ==============================
[pointed] Nose (0) => prob: 0.77942 / x: 77 / y: 70
[pointed] Neck (1) => prob: 0.77348 / x: 85 / y: 126
[pointed] RShoulder (2) => prob: 0.66136 / x: 46 / y: 126
[pointed] RElbow (3) => prob: 0.43241 / x: 23 / y: 220
[not pointed] RWrist (4) => prob: 0.04590 / x: 23 / y: 220
[pointed] LShoulder (5) => prob: 0.60878 / x: 124 / y: 126
[pointed] LElbow (6) => prob: 0.67931 / x: 147 / y: 220
[pointed] LWrist (7) => prob: 0.60880 / x: 132 / y: 157
[not pointed] RHip (8) => prob: 0.18156 / x: 62 / y: 252
[not pointed] RKnee (9) => prob: 0.00294 / x: 23 / y: 228
[not pointed] RAnkle (10) => prob: 0.00298 / x: 15 / y: 236
[not pointed] LHip (11) => prob: 0.16625 / x: 116 / y: 252
[not pointed] LKnee (12) => prob: 0.00380 / x: 116 / y: 212
[not pointed] LAnkle (13) => prob: 0.00154 / x: 140 / y: 212
[pointed] REye (14) => prob: 0.84003 / x: 70 / y: 63
[pointed] LEye (15) => prob: 0.87403 / x: 85 / y: 63
[not pointed] REa


============================== COCO Model ==============================
[pointed] Nose (0) => prob: 0.75043 / x: 77 / y: 70
[pointed] Neck (1) => prob: 0.75669 / x: 85 / y: 126
[pointed] RShoulder (2) => prob: 0.69168 / x: 46 / y: 126
[pointed] RElbow (3) => prob: 0.60089 / x: 0 / y: 196
[pointed] RWrist (4) => prob: 0.75995 / x: 46 / y: 181
[pointed] LShoulder (5) => prob: 0.66094 / x: 124 / y: 126
[pointed] LElbow (6) => prob: 0.73349 / x: 155 / y: 212
[pointed] LWrist (7) => prob: 0.82366 / x: 124 / y: 157
[not pointed] RHip (8) => prob: 0.19250 / x: 54 / y: 252
[not pointed] RKnee (9) => prob: 0.00305 / x: 7 / y: 204
[not pointed] RAnkle (10) => prob: 0.00228 / x: 31 / y: 173
[pointed] LHip (11) => prob: 0.20069 / x: 116 / y: 252
[not pointed] LKnee (12) => prob: 0.00945 / x: 108 / y: 212
[not pointed] LAnkle (13) => prob: 0.00385 / x: 23 / y: 204
[pointed] REye (14) => prob: 0.86065 / x: 70 / y: 55
[pointed] LEye (15) => prob: 0.80737 / x: 85 / y: 63
[not pointed] REar (16) => p

In [50]:
#print(point_dict)

## Down here is codes for testing. Delete it or do not execute. ##

In [30]:

point_temp = []
asdf = list(str(list(BODY_PARTS_COCO.values())[i])+'_'+str(points[i]) for i in point_indexes)
for i, j in zip(points, BODY_PARTS_COCO.values()):
    if points.index(i) in point_indexes:
        print(i, j)
        point_temp.append(str(j)+'_'+str(i))

(77, 70) Nose
(85, 126) Neck
(46, 126) RShoulder
(7, 196) RElbow
(46, 181) RWrist
(124, 126) LShoulder
(155, 212) LElbow
(124, 157) LWrist
(70, 55) REye
(85, 63) LEye
(101, 70) LEar


In [27]:
pointstring = "  ".join(point_temp)
f = open("point_test.txt", 'w')
f.write(pointstring)
f.close()

In [32]:
print(asdf, pointstring)

['Nose_(77, 70)', 'Neck_(85, 126)', 'RShoulder_(46, 126)', 'RElbow_(7, 196)', 'RWrist_(46, 181)', 'LShoulder_(124, 126)', 'LElbow_(155, 212)', 'LWrist_(124, 157)', 'REye_(70, 55)', 'LEye_(85, 63)', 'REar_None', 'LEar_(101, 70)'] Nose_(77, 70)  Neck_(85, 126)  RShoulder_(46, 126)  RElbow_(7, 196)  RWrist_(46, 181)  LShoulder_(124, 126)  LElbow_(155, 212)  LWrist_(124, 157)  REye_(70, 55)  LEye_(85, 63)  LEar_(101, 70)
